## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

In [3]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [4]:
from google.colab import drive
drive.mount('/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [19]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [8]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [10]:
data_trunc = data.iloc[0:1000,:]

In [11]:
data_trunc.shape

(1000, 5)

In [12]:
data_trunc['volume'] = data_trunc['volume']/1000000

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
data_trunc.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [20]:
from sklearn.model_selection import train_test_split

In [15]:
X = data_trunc.drop(['volume'],axis=1)
Y = data_trunc[['volume']]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

#### Convert Training and Test Data to numpy float32 arrays


In [17]:
X_train = X_train.astype('float32')

In [18]:
X_test = X_test.astype('float32')

In [19]:
y_train = y_train.astype('float32')

In [20]:
y_test= y_test.astype('float32')

In [21]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 670 entries, 508 to 175
Data columns (total 4 columns):
open     670 non-null float32
close    670 non-null float32
low      670 non-null float32
high     670 non-null float32
dtypes: float32(4)
memory usage: 15.7 KB
None


In [22]:
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 778 to 191
Data columns (total 4 columns):
open     330 non-null float32
close    330 non-null float32
low      330 non-null float32
high     330 non-null float32
dtypes: float32(4)
memory usage: 7.7 KB
None


In [23]:
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 670 entries, 508 to 175
Data columns (total 1 columns):
volume    670 non-null float32
dtypes: float32(1)
memory usage: 7.9 KB
None


In [24]:
print(y_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 778 to 191
Data columns (total 1 columns):
volume    330 non-null float32
dtypes: float32(1)
memory usage: 3.9 KB
None


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
scaler = StandardScaler()

In [27]:
scaler.fit(X_train)
X_train_Scale = scaler.transform(X_train)

In [28]:
scaler.fit(X_test)
X_test_Scale = scaler.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [29]:
#W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
#b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

In [30]:
W = tf.random_normal(shape=[4,1])
b = tf.random_normal(shape=[1])

2.Define a function to calculate prediction

In [31]:
def prediction(x, w, b):  
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    return y

3.Loss (Cost) Function [Mean square error]

In [32]:
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)  
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [33]:
def train(x, y_actual, w, b, learning_rate=0.01):
    #Record math ops on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss w.r.t Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [34]:
X_train_Scale = X_train_Scale.astype("float32")
X_test_Scale = X_test_Scale.astype("float32")

In [35]:
for i in range(100):
    W, b = train(X_train_Scale, np.array(y_train), W, b)
    print('Current Loss on iteration', i, loss(np.array(y_train), prediction(X_train_Scale, W, b)).numpy())

Current Loss on iteration 0 358.6415
Current Loss on iteration 1 352.4795
Current Loss on iteration 2 347.04562
Current Loss on iteration 3 342.23666
Current Loss on iteration 4 337.96506
Current Loss on iteration 5 334.15616
Current Loss on iteration 6 330.74655
Current Loss on iteration 7 327.68228
Current Loss on iteration 8 324.91742
Current Loss on iteration 9 322.41263
Current Loss on iteration 10 320.13458
Current Loss on iteration 11 318.05475
Current Loss on iteration 12 316.1485
Current Loss on iteration 13 314.39523
Current Loss on iteration 14 312.77682
Current Loss on iteration 15 311.2779
Current Loss on iteration 16 309.88513
Current Loss on iteration 17 308.58728
Current Loss on iteration 18 307.37442
Current Loss on iteration 19 306.23798
Current Loss on iteration 20 305.17062
Current Loss on iteration 21 304.16595
Current Loss on iteration 22 303.21832
Current Loss on iteration 23 302.32266
Current Loss on iteration 24 301.47495
Current Loss on iteration 25 300.67123


### Get the shapes and values of W and b

In [36]:
W, b = train(X_train_Scale, np.array(y_train), W, b)
print ('W',W,'b',b)

W tf.Tensor(
[[-1.9768025 ]
 [-0.00859479]
 [ 1.6261301 ]
 [-1.309212  ]], shape=(4, 1), dtype=float32) b tf.Tensor([4.780836], shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [37]:
ypr = prediction(X_test_Scale, W, b).numpy()
print(ypr)

[[  4.9950666 ]
 [  4.8976674 ]
 [  5.8034115 ]
 [  5.6566315 ]
 [  2.965839  ]
 [  4.0135922 ]
 [  6.2328987 ]
 [  2.842577  ]
 [  4.73297   ]
 [  2.7812924 ]
 [  3.260041  ]
 [  2.9324331 ]
 [  5.582356  ]
 [  5.117134  ]
 [  5.0409684 ]
 [  4.969188  ]
 [  2.9722657 ]
 [  5.337561  ]
 [  5.7084603 ]
 [  1.3478212 ]
 [  2.844407  ]
 [  3.3973298 ]
 [  5.4924088 ]
 [  5.4241595 ]
 [  4.1532154 ]
 [  5.916912  ]
 [  3.3235357 ]
 [  5.7080355 ]
 [  4.7743473 ]
 [  6.4100633 ]
 [  5.2725873 ]
 [  3.2046707 ]
 [  5.919631  ]
 [  5.1803656 ]
 [  6.446301  ]
 [  5.614757  ]
 [  2.9023087 ]
 [  5.5805254 ]
 [  5.3345838 ]
 [  3.0312715 ]
 [  6.2002807 ]
 [-12.62406   ]
 [  4.8936977 ]
 [  5.570632  ]
 [  6.062438  ]
 [  5.2876606 ]
 [  5.786396  ]
 [  6.299057  ]
 [  5.9155955 ]
 [  2.821558  ]
 [  6.538802  ]
 [  4.747223  ]
 [  6.117157  ]
 [  3.2708955 ]
 [  2.9731395 ]
 [  6.6215124 ]
 [  5.8693485 ]
 [  6.1028733 ]
 [  6.618039  ]
 [  5.4325495 ]
 [  5.05936   ]
 [  5.7453203 ]
 [  5.71

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

In [21]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

### Load the given Iris data using pandas (Iris.csv)

In [22]:
iris_df = pd.read_csv('Iris.csv')

In [23]:
iris_df.drop('Id',axis=1,inplace=True)

In [24]:
iris_df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [25]:
le = LabelEncoder()
le.fit(iris_df['Species'])
iris_df['Species'] = le.transform(iris_df['Species'])

In [26]:
iris_df[['out1','out2','out3']] = pd.get_dummies(iris_df['Species'])

In [27]:
iris_df.drop('Species',axis=1,inplace=True)

In [28]:
iris_df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,out1,out2,out3
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [29]:
X = iris_df.iloc[:,0:-3]

In [30]:
Y = iris_df.iloc[:,-3:]

In [31]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [32]:
Y.head()

,out1,out2,out3
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


### Divide the dataset into Training and test (70:30)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [36]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
#model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Model Training 

In [39]:
model.fit(X_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
105/105 [==============================] - 0s 190us/step - loss: 0.0825 - acc: 0.9714
Epoch 2/150
105/105 [==============================] - 0s 114us/step - loss: 0.0791 - acc: 0.9810
Epoch 3/150
105/105 [==============================] - 0s 76us/step - loss: 0.0845 - acc: 0.9810
Epoch 4/150
105/105 [==============================] - 0s 86us/step - loss: 0.0790 - acc: 0.9810
Epoch 5/150
105/105 [==============================] - 0s 124us/step - loss: 0.0854 - acc: 0.9714
Epoch 6/150
105/105 [==============================] - 0s 105us/step - loss: 0.1113 - acc: 0.9333
Epoch 7/150
105/105 [==============================] - 0s 114us/step - loss: 0.0857 - acc: 0.9810
Epoch 8/150
105/105 [==============================] - 0s 105us/step - loss: 0.0822 - acc: 0.9714
Epoch 9/150
105/105 [==============================] - 0s 114us/step - loss: 0.0834 - acc: 0.9714
Epoch 10/150
105/105 [==============================] - 0s 114us/step - loss: 0.0858 - acc: 0.9714
Epoch 11/150
105/105 

105/105 [==============================] - 0s 86us/step - loss: 0.0753 - acc: 0.9714
Epoch 85/150
105/105 [==============================] - 0s 105us/step - loss: 0.0702 - acc: 0.9810
Epoch 86/150
105/105 [==============================] - 0s 95us/step - loss: 0.0815 - acc: 0.9619
Epoch 87/150
105/105 [==============================] - 0s 76us/step - loss: 0.0721 - acc: 0.9810
Epoch 88/150
105/105 [==============================] - 0s 86us/step - loss: 0.0731 - acc: 0.9714
Epoch 89/150
105/105 [==============================] - 0s 105us/step - loss: 0.0848 - acc: 0.9714
Epoch 90/150
105/105 [==============================] - 0s 105us/step - loss: 0.0746 - acc: 0.9619
Epoch 91/150
105/105 [==============================] - 0s 114us/step - loss: 0.0771 - acc: 0.9810
Epoch 92/150
105/105 [==============================] - 0s 105us/step - loss: 0.0801 - acc: 0.9810
Epoch 93/150
105/105 [==============================] - 0s 105us/step - loss: 0.0839 - acc: 0.9714
Epoch 94/150
105/105 [=====

### Model Prediction

In [46]:
from sklearn import metrics

In [59]:
y_predict=model.predict_classes(X_test)

In [60]:
y_predict

array([2, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 0, 2, 0, 1, 2, 2, 0, 0, 1, 2,
       1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 0, 2, 1, 0, 0, 0, 0, 2, 2, 1, 2, 2,
       1], dtype=int64)

### Save the Model

In [61]:
model.save('iris_model.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [62]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [63]:
model.fit(X_test, y_test, epochs=150, batch_size=10)

Epoch 1/150
45/45 [==============================] - 0s 4ms/step - loss: 1.0342 - acc: 0.5111
Epoch 2/150
45/45 [==============================] - 0s 133us/step - loss: 0.8251 - acc: 0.5778
Epoch 3/150
45/45 [==============================] - 0s 222us/step - loss: 0.7680 - acc: 0.5333
Epoch 4/150
45/45 [==============================] - 0s 133us/step - loss: 0.7466 - acc: 0.5556
Epoch 5/150
45/45 [==============================] - 0s 200us/step - loss: 0.6995 - acc: 0.7111
Epoch 6/150
45/45 [==============================] - 0s 133us/step - loss: 0.7176 - acc: 0.6000
Epoch 7/150
45/45 [==============================] - 0s 200us/step - loss: 0.6700 - acc: 0.6222
Epoch 8/150
45/45 [==============================] - 0s 156us/step - loss: 0.6813 - acc: 0.6000
Epoch 9/150
45/45 [==============================] - 0s 200us/step - loss: 0.6932 - acc: 0.6667
Epoch 10/150
45/45 [==============================] - 0s 156us/step - loss: 0.6033 - acc: 0.7778
Epoch 11/150
45/45 [=====================

45/45 [==============================] - 0s 178us/step - loss: 0.2973 - acc: 0.9111
Epoch 86/150
45/45 [==============================] - 0s 178us/step - loss: 0.2791 - acc: 0.9111
Epoch 87/150
45/45 [==============================] - 0s 111us/step - loss: 0.2490 - acc: 0.9333
Epoch 88/150
45/45 [==============================] - 0s 156us/step - loss: 0.2575 - acc: 0.9333
Epoch 89/150
45/45 [==============================] - 0s 156us/step - loss: 0.2553 - acc: 0.9333
Epoch 90/150
45/45 [==============================] - 0s 200us/step - loss: 0.2503 - acc: 0.9111
Epoch 91/150
45/45 [==============================] - 0s 156us/step - loss: 0.2366 - acc: 0.9111
Epoch 92/150
45/45 [==============================] - 0s 156us/step - loss: 0.2171 - acc: 0.9778
Epoch 93/150
45/45 [==============================] - 0s 244us/step - loss: 0.2395 - acc: 0.9111
Epoch 94/150
45/45 [==============================] - 0s 156us/step - loss: 0.2223 - acc: 0.9333
Epoch 95/150
45/45 [=======================

In [64]:
y_predict=model.predict_classes(X_test)

In [65]:
y_predict

array([2, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 0, 2, 0, 1, 2, 2, 0, 0, 1, 2,
       1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 0, 2, 1, 0, 0, 0, 0, 2, 2, 1, 2, 2,
       1], dtype=int64)